In [ ]:
#This scipt is for reading momentum surfaces in a Tropical Cyclone

In [1]:
#input necessary packages here

#For Map
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.ticker as mticker 

#wrf packages
import wrf
from wrf import (to_np, interplevel, geo_bounds, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, getvar, CoordPair, vertcross, ll_to_xy, xy_to_ll)

#to read in data
import glob
import matplotlib.pyplot as plt

#for some fun math
import math
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime
from netCDF4 import Dataset, num2date, date2num
from scipy.ndimage import gaussian_filter

In [2]:
#only run the cell below for the 1.33 kilometer inner nest

#Function Below is for a storm following map projection. Credit: Dr. Sharanya Majumdar @The University of Miami
dataproj = ccrs.PlateCarree()

def create_map_background(tclon,tclat):

    tclon5a=5*(int(tclon/5)-1)
    tclon5b=5*(int(tclon/5))
    tclat5a=5*(int(tclat/5)-1)
    tclat5b=5*(int(tclat/5))
     
    #Specify plot size
    fig=plt.figure(figsize=(15, 15))
    ax=plt.subplot(111, projection=dataproj)
    
    #specify boundary extent from TC center. Current specifications give us a 10° x 10° projection
    ax.set_extent([tclon-4, tclon+4, tclat-4, tclat+4],ccrs.PlateCarree()) #Constrain FOV of plot
    gl = ax.gridlines(color='gray',alpha=0.5,draw_labels=True)
    gl.xlabels_top, gl.ylabels_right = False, False
    gl.xlabel_style, gl.ylabel_style = {'fontsize': 16}, {'fontsize': 16}
    gl.xlocator = mticker.FixedLocator([tclon5a-10, tclon5a-5, tclon5a, tclon5b, tclon5b+5, tclon5b+10])
    gl.ylocator = mticker.FixedLocator([tclat5a-10, tclat5a-5, tclat5a, tclat5b, tclat5b+5, tclat5b+10])
    gl.xformatter = LongitudeFormatter(zero_direction_label=True)
    gl.yformatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(gl.xformatter)
    ax.yaxis.set_major_formatter(gl.yformatter)
    return fig, ax

In [3]:
#First, Specifiy File path
filepath = '/glade/work/cmasiello/Control_Run/wrfout/Control_d03/wrfout_d03*'  #For ncar files
datafiles = sorted(glob.glob(filepath))

In [4]:
# set some constants for later use that we may or may not use, just there for any meteorological claculation you may conduct
deltap     = 5000.        # vertical isobaric spacing
resol      = 0.5          # degrees
pref       = 100000.      # Pa
cv         = 717.         # J kg-1 K-1
cp         = 1004.7       # J kg-1 K-1
omega      = 7.292e-5     # Rotation rate s-1       
rd         = 287.04       # J kg-1 K-1
fo         = 1.0e-4       # s-1; assume constant Coriolis parameter (Eliassen 1962)
po         = 100000.      # standard pressure (Pa)
r_earth    = 6.3781e6     # meters (Earth Radius)
grav       = 9.80665      # m/s-2
d2r        = (np.pi)/180. # degrees to radians
rv         = 461.50
k          = 0.4          #Von Karmen Constant

def find_nearest(a, val):
    return np.abs(a - val).argmin()

In [5]:
#Below is code needed to interpolate values on to pressure levels

#Pressure levels we are considering for our aziumuthal averages
plevs  = [100000., 99000., 98000., 97000., 96000., 95000., 94000., 93000., 92000., 91000.,
           90000., 89000., 88000., 87000., 86000., 85000., 84000., 83000., 82000., 81000., 80000.,
           77500., 75000., 72500., 70000., 67500., 65000., 62500., 60000., 57500., 55000., 52500.,
           50000., 47500., 45000., 42500., 40000., 37500., 35000., 32500., 30000., 27500., 25000.,
           22500., 20000., 17500., 15000., 12500., 10000., 7500., 6000.]

nlevs  =len(plevs) #level of interest is 850 hPa, which is the sixth level

print("Number of Pressure levels in the vertical", nlevs)
yplabs = ['']*nlevs

Number of Pressure levels in the vertical 51


In [ ]:
#Below is the code we will use to calculate momentum surfaces

#for z in range(1, len(datafiles)):
for z in range(15,16):
    ncfile = Dataset(datafiles[z])
    wrf_out_data = xr.open_dataset(datafiles[z])   
    Time = wrf.extract_times(ncfile, timeidx=0, method='cat', squeeze=True, cache=None, meta=False, do_xtime=False)
    timestr=(str(Time))

    #for plotting
    titletime=(timestr[0:10]+' '+timestr[11:16])
    filetime= (timestr[0:10]+'_'+timestr[11:13])

    #Next, isolate the variable we care to look at, in this case we will only care about the pressure pertubations
    P_pertubation = wrf_out_data['P']
    surface_P_pertubation = P_pertubation[0,0,:,:]
    
    # Apply a Gaussian filter to smooth the pressure field
    smoothed_pressure = gaussian_filter(surface_P_pertubation, sigma=10)  # Adjust sigma as necessary

    # Now find the minimum pressure in the smoothed field
    minpressure_smoothed = np.min(smoothed_pressure)
    print(minpressure_smoothed)
    minp_smoothed = minpressure_smoothed

    # Get the index of the minimum pressure
    p_index_smoothed = np.argwhere(smoothed_pressure == minp_smoothed)
    s_n_smoothed = p_index_smoothed[0][0]
    w_e_smoothed = p_index_smoothed[0][1]

    # Update the storm center position
    ypos = s_n_smoothed
    xpos = w_e_smoothed

    tc_lon = wrf_out_data['XLONG'][0,s_n_smoothed,w_e_smoothed].values
    tc_lat = wrf_out_data['XLAT'][0,s_n_smoothed,w_e_smoothed].values

    #fig, ax = create_map_background(tc_lon,tc_lat) #We are only looking at 11 indices, and our list goes from 0-11, not 24 and up

    #################################################################################################################################################
    #now that we have center information, we can do momentum calculation

    #constants
    lats = getvar(ncfile, "XLAT")
    lons = getvar(ncfile, "XLONG") 
    land = getvar(ncfile, "LANDMASK")
    ptop = getvar(ncfile, "P_TOP")

    nx = lats.shape[0]
    ny = lats.shape[1]
    nz = P_pertubation.shape[1]

    #variables we will need for momentum calculations
    # Get variables 
    z = getvar(ncfile, "z")
    p = getvar(ncfile, "pressure")
    ua = getvar(ncfile, "ua")
    va = getvar(ncfile, "va")
    f = wrf_out_data['F'][0,:,:]

    ugrd, vgrd, zgrd, pgrd = [np.zeros([nx,ny,nlevs]) for _ in range(4)]  #Range is just the number of variables specified to the left

    for k in range (nlevs):
        clev = plevs[k]/100.
        yplabs[k] = clev
        ##print('interpolating to: ',clev)

        # Do vertical interpolation to specified pressure surfaces (in hPa)
        uap   = interplevel(ua, p, clev)
        vap   = interplevel(va, p, clev)
        z_new = interplevel(z, p, clev)
        p_new = interplevel(p, p, clev)

        # Smoothing factor, redundat step from old scripts, but if you would want to apply smoothing you would do that here
        # smfac = 3   # light smoothing, none for reflectivity
        #smoothing example: ugrd[:,:,k] = mpcalc.smooth_gaussian(uap[:,:],smfac)
        ugrd[:,:,k] = uap[:,:]
        vgrd[:,:,k] = vap[:,:]
        zgrd[:,:,k] = z_new[:,:]
        pgrd[:,:,k] = p_new[:,:]
        

    #First, we have to create lists
    ydist, xdist, tcdist, rlonfac, theta_ang = [np.zeros([nx,ny]) for _ in range(5)]
    u_radial, v_azimuth, momentum            = [np.zeros([nx,ny,nlevs]) for _ in range(3)]
    
    # compute distance from each grid point to storm center
    rlonfac[:,:] = 111000.*np.cos(d2r*lats[:,:])
    ydist[:,:]   = (lats[:,:] - lats[ypos,xpos])*111000
    xdist[:,:]   = (lons[:,:] - (lons[ypos,xpos]))*rlonfac
    tcdist[:,:]  = (xdist[:,:]**2 + ydist[:,:]**2)**0.5
    
    #cvalculate angle
    theta_ang = np.arctan2(ydist, xdist)

    # Pre-calculate cos and sin of theta_ang for use in vectorized calculations
    cos_theta = np.cos(theta_ang)
    sin_theta = np.sin(theta_ang)
    
    for k in range(nlevs):
        #print("The number loop we are in is", k)
        # Vectorized calculation of u_radial and v_azimuth
        u_radial[:,:,k] = ugrd[:,:,k]*cos_theta + vgrd[:,:,k]*sin_theta
        v_azimuth[:,:,k] = -ugrd[:,:,k]*sin_theta + vgrd[:,:,k]*cos_theta
        
        # Vectorized calculation of momentum
        momentum[:,:,k] = tcdist * v_azimuth[:,:,k] + (1/2) * f * (tcdist**2)

    #now, we are doing west/east cross sections so only care about the value of our longitude beginning and end point, 
    lon_beginning = tc_lon - 4 #4 degrees lon to the left of the TC center
    lon_ending    = tc_lon + 4 #4 degrees lon to the right of the TC center

    start_point = CoordPair(lat = tc_lat, lon = lon_beginning )
    end_point = CoordPair(lat = tc_lat, lon = lon_ending)

    #transpose momentum array into the correct shape 
    momentum_transposed = np.transpose(momentum, (2, 0, 1))
    p_transposed =  np.transpose(pgrd, (2, 0, 1))
    
    print("Momentum shape:", momentum_transposed.shape)
    print("Z shape:", z_transposed.shape)

    # Convert start and end point lat/lon to grid indices
    start_x, start_y = ll_to_xy(ncfile, start_point.lat, start_point.lon)
    end_x, end_y = ll_to_xy(ncfile, end_point.lat, end_point.lon)

    #Verify Information is what we want
    print(start_x.values, end_x.values, start_y.values)

    cross_section_momentum = momentum_transposed[:, start_y.values, start_x.values:end_x.values + 1]

    # Mask the NaN values
    masked_cross_section_momentum = np.ma.masked_invalid(cross_section_momentum)

    vertical_levels = np.arange(masked_cross_section_momentum.shape[0])  # Your vertical levels (51 levels in this case)
    horizontal_extent = np.arange(masked_cross_section_momentum.shape[1])  # Your horizontal extent (581 points in this case)

    #print("lons shape", lons.shape)
    # Create an array of longitude values corresponding to the cross-section
    cross_section_lons = lons[start_y.values, start_x.values:end_x.values + 1]
    cross_section_lons = cross_section_lons.squeeze()
    
    # Verify the shape
    #print("cross_section_lons shape:", cross_section_lons.shape)

    # Assuming pressure is in hPa and we have pressure levels for the Y-axis
    Y = np.array(plevs) / 100.0  # Convert from Pa to 

    # X is 1D, representing the longitude values across the cross-section
    X = cross_section_lons[:]
    
    # Assuming Y is the pressure values obtained as shown in previous steps
    X2D, Y2D = np.meshgrid(cross_section_lons,  Y)
    
    # Plot the data
    fig, ax = plt.subplots(figsize=(24, 12))
    contourf = ax.contourf(X2D, Y2D, masked_cross_section_momentum, levels = np.arange(0,2.25e7,.25e7), cmap='viridis', extend='both')
    
    # Adding a colorbar
    cbar = fig.colorbar(contourf)
    cbar.set_label(r'Momentum ($m^2/s$)')
    
    # Set axis labels
    ax.invert_yaxis()
    ax.set_xlabel('Longitude (°W)', fontsize=18)
    ax.set_ylabel('Pressure (hPa)', fontsize=18)
    
    # Title
    title = f'Momentum Cross-section at {tc_lat:.2f}° Lat'
    plt.title(title,loc = 'left', fontsize=18)
    plt.title("By: Cameron Masiello" ,loc = 'right', fontsize=18)
    
    # Show plot
    plt.show()

-9666.736
